# Information about Josh's turbulent sphere initial conditions

2020 January 31 - Feb 01, Feb 06

In paper one (Wall et al. 2019, ApJ 887:62) and paper two (2020 January draft), Josh reports the following runs:

<img src="wall_paper_one_runs.png" width="600">

<img src="wall_paper_two_runs.png" width="600">

These correspond to, on Draco and Cartesius:

    M3   -> Draco_CartM3V02A
    M3f  -> old_M3V02B_same_cloud_from_M3V02A
    M3f2 -> Draco_M3V02B
    M4f -> M4V02AW          (0000 file created May 26, 2018)
    M5f -> M5V02AW_restart  (0000 file [restart] created June 18, 2018)
    
Note that the M5 run started from a different folder:

    M5V02AW/M5V02AW_bad_IGIMF_start  (0000 file created April 28, 2018)

Josh's most recent simulation runs (started from scratch) are:

    M3V02BWL10_USM  (started Sep 1 2018)
    M4V02BWL10_USM  (started Sep 1 2018)
    M5V02AWL10_USM  (started Sep 1 2018)


In [ ]:
from __future__ import division, print_function

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from os import path

import yt
yt.funcs.mylog.setLevel(30)  # 10=DEBUG, 20=INFO, 30=WARNING, 40=ERROR, 50=CRITICAL

#from bang import FlashRun
from ytplotlib import plot_ray_field

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Configuration - load sphere / dataset to plot

Careful... there are decent number of recurring "global" variables

For M3 runs, use the cubes obtained by Sean from Draco, ompare to Cartesius runs that /appear/ to be based on the same initial conditions.

For M5 runs, use Cartesius 0000 plot file, compare to "mimic" cube that is probably NOT the same initial condition.

In [ ]:
# M3 First attempt using cartesius data only compared to my home-made mimic sphere
#INITIAL_PLT = "/rigel/home/at3222/sf/joshrun/M3V02BWL10_USM/turbsph_hdf5_plt_cnt_0000"
#CUBE = "/rigel/astro/users/at3222/sf/run_bin/josh_sphere_mimics/M3V02"
#RADIUS = 5
#BOXHW = 7

# Use cube provided by sean
#INITIAL_PLT = "/rigel/home/at3222/sf/joshrun/M3V02AW_USM/turbsph_hdf5_plt_cnt_0000"  # NOTE change from BWL10 to AW
#CUBE = "/rigel/astro/users/at3222/sf/joshrun/cubeM3V02A"
#RADIUS = 3
#BOXHW = 5

# Use cube provided by sean
#INITIAL_PLT = "/rigel/home/at3222/sf/joshrun/M3V02BWL10_USM/turbsph_hdf5_plt_cnt_0000"
#CUBE = "/rigel/astro/users/at3222/sf/joshrun/cubeM3V02B"
#RADIUS = 5
#BOXHW = 7

# M5 Cartesius data compared to my home-made mimic sphere
INITIAL_PLT = "/rigel/home/at3222/sf/joshrun/M5V02AW_bad_IGIMF_start_turbsph_hdf5_plt_cnt_0000"
CUBE = "/rigel/astro/users/at3222/sf/run_bin/josh_sphere_mimics/M5V02"
RADIUS = 50
BOXHW = 55

cmpc = 3.0856775809623245e+18  # 1 parsec in cm; yt value
Msun = 1.98841586e+33  # solar mass in g; yt value

In [ ]:
started = datetime.now()
print("loading", INITIAL_PLT)
ds = yt.load(INITIAL_PLT)
ad = ds.all_data()
print("done loading, elapsed", datetime.now() - started)

NREF = ds.parameters['lrefine_min']

started = datetime.now()
print("loading", CUBE)
dat = np.loadtxt(CUBE)
print("done loading, elapsed", datetime.now() - started)

NBLOCK = 128

assert NBLOCK**3 == dat.shape[0]
cube_rho = np.reshape(dat[:,3], (NBLOCK,NBLOCK,NBLOCK))
cube_vx = np.reshape(dat[:,5], (NBLOCK,NBLOCK,NBLOCK))
cube_vy = np.reshape(dat[:,6], (NBLOCK,NBLOCK,NBLOCK))
cube_vz = np.reshape(dat[:,7], (NBLOCK,NBLOCK,NBLOCK))

# Check mass and density profiles.  Compare to analytic Gaussian with expected FWHM.

In [ ]:
def report_enclosed_mass(ds, ad, dat):
    """What it says on the tin"""
    
    # https://yt-project.org/doc/cookbook/calculating_information.html
    mtot = ad.quantities.total_quantity(["cell_mass"]).in_units('g').value / 1.99e33  # solar mass in g
    sph = ds.sphere([0,0,0], (RADIUS, "pc"))
    msph = sph.quantities.total_quantity(["cell_mass"]).in_units('g').value / 1.99e33  # solar mass in g
    dens_max = ad.quantities.extrema('dens')[-1].in_units('g/cm**3')

    print(INITIAL_PLT)
    print("Total mass (Msun):", mtot)
    print("Mass enclosed in {} pc (Msun):".format(RADIUS), msph)
    print("dens extrema", dens_max)
    
    cube_total = np.sum(cube_rho) * (2*BOXHW*cmpc/NBLOCK)**3  # g
    cube_amb = np.amin(cube_rho) * (2*BOXHW*cmpc)**3 * (1 - 4*np.pi/3*(RADIUS*cmpc)**3/(2*BOXHW*cmpc)**3)

    print()
    print(CUBE)
    print("Total mass (Msun) =".format(CUBE), cube_total / Msun)
    print("Total - ambient mass (Msun):", (cube_total - cube_amb) / Msun)  # Note that we are using analytic formula here!!!!
    print("dens extrema", np.amax(cube_rho))

In [ ]:
report_enclosed_mass(ds, ad, dat)

In [ ]:
def plotfile_density(ds, ax):
    """Plot the density from plotfile"""
    xmin = ds.domain_left_edge.in_units('pc')[0]
    xmax = ds.domain_right_edge.in_units('pc')[0]
    r1 = yt.YTArray([xmin, 0, 0], input_units='kpc')
    r2 = yt.YTArray([xmax, 0, 0], input_units='kpc')
    r = ds.ray(r1, r2)
    plt.sca(ax)
    plot_ray_field(plt.gca(), r, 'density', xunit='pc', yunit='g/cm**3', fmt='.-', markersize=3, alpha=0.65, label='plt_cnt_0000')

def predict_density(ds, ax):
    """Plot the predicted density, but using central density from simulation file"""
    dens_max = ad.quantities.extrema('dens')[-1].in_units('g/cm**3')
    x = np.linspace(-BOXHW, +BOXHW, 1000)  # probably not exact, beware
    y = dens_max * np.exp( -1*x**2/RADIUS**2 )
    ax.plot(x + BOXHW, y, '-k', linewidth=3, alpha=0.5, label='predict', zorder=-99)

def cube_density(rho, ax):
    x = np.linspace(-BOXHW, +BOXHW, num=NBLOCK)  # probably not exact, beware
    ax.plot(x + BOXHW, rho[:,int(NBLOCK/2),int(NBLOCK/2)], '.-', markersize=3, alpha=0.65, label='mimic 128^3')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,6))
for ax in axes:
    predict_density(ds, ax)
    plotfile_density(ds, ax)
    cube_density(cube_rho, ax)
    plt.xlabel('x (pc)')
    plt.ylabel('density (g/cm^3)')

axes[0].legend()
axes[1].set_yscale('log')
plt.suptitle("density slice across x axis" + '\n' + INITIAL_PLT + '\n' + CUBE)
plt.subplots_adjust(top=0.85)
plt.show()

# Plot and calculate volume-weighted velocity amplitude

In [ ]:
def plotfile_vel(ds, ax, which='velx'):
    """Plot the density from plotfile"""
    xmin = ds.domain_left_edge.in_units('pc')[0]
    xmax = ds.domain_right_edge.in_units('pc')[0]
    r1 = yt.YTArray([xmin, 0, 0], input_units='kpc')
    r2 = yt.YTArray([xmax, 0, 0], input_units='kpc')
    r = ds.ray(r1, r2)
    plt.sca(ax)
    plot_ray_field(plt.gca(), r, which, xunit='pc', yunit='km/s', fmt='.-', markersize=3, alpha=0.65, label='plt_cnt_0000 '+which)


def cube_vel(cube_vel, ax, which='velx'):
    x = np.linspace(-BOXHW, +BOXHW, num=NBLOCK)  # probably not exact, beware
    ax.plot(x + BOXHW, cube_vel[:,int(NBLOCK/2),int(NBLOCK/2)] / 1e5,  # CONVERT to km/s
            '.-', markersize=3, alpha=0.65, label='128^3 array '+which)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,6), sharex=True, sharey=True)

plotfile_vel(ds, axes[0], which='velx')
plotfile_vel(ds, axes[0], which='vely')
plotfile_vel(ds, axes[0], which='velz')
cube_vel(cube_vx, axes[1], which='velx')
cube_vel(cube_vy, axes[1], which='vely')
cube_vel(cube_vz, axes[1], which='velz')

for ax in axes:
    ax.set_xlabel('x (pc)')
    ax.set_ylabel('velocity (km/s)')

axes[0].legend()
axes[1].legend()
plt.suptitle("vel{x,y,z} slice across x axis" + '\n' + INITIAL_PLT + '\n' + CUBE)
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
def report_mean_velocity_by_cell(ds, cover):
    
    xmin = ds.domain_left_edge.to('pc')[0].d
    xmax = ds.domain_right_edge.to('pc')[0].d

    sphcsel = np.sqrt(cover['x']**2 + cover['y']**2 + cover['z']**2).to('pc') < RADIUS

    velx_rms = np.mean(cover['velx'][sphcsel]**2)**0.5
    vely_rms = np.mean(cover['vely'][sphcsel]**2)**0.5
    velz_rms = np.mean(cover['velz'][sphcsel]**2)**0.5
    vel_rms = np.mean(cover['velx'][sphcsel]**2+cover['vely'][sphcsel]**2+cover['velz'][sphcsel]**2)**0.5
    
    print("    vx,rms", velx_rms.to('km/s'))
    print("    vy,rms", vely_rms.to('km/s'))
    print("    vz,rms", velz_rms.to('km/s'))
    print("    v,rms", vel_rms.to('km/s'))

def report_mean_velocity_by_cell_cube(vx, vy, vz):
    
    ax = np.linspace(-BOXHW, BOXHW, NBLOCK)
    ay = np.linspace(-BOXHW, BOXHW, NBLOCK)
    az = np.linspace(-BOXHW, BOXHW, NBLOCK)
    xx,yy,zz = np.meshgrid(ax,ay,az)
    rr = (xx*xx+yy*yy+zz*zz)**0.5
    sphcubesel = rr < RADIUS

    print("    vx,rms", np.mean(vx[sphcubesel]**2)**0.5 / 1e5)  # convert to km/s
    print("    vy,rms", np.mean(vy[sphcubesel]**2)**0.5 / 1e5)  # convert to km/s
    print("    vz,rms", np.mean(vz[sphcubesel]**2)**0.5 / 1e5)  # convert to km/s
    print("    v,rms", np.mean(vx[sphcubesel]**2+vy[sphcubesel]**2+vz[sphcubesel]**2)**0.5 / 1e5)  # convert to km/s

In [ ]:
cover = ds.covering_grid(level=NREF-1, left_edge=ds.domain_left_edge,
                         dims=ds.domain_dimensions*2**(NREF-1))

print("plt_cnt_0000,", INITIAL_PLT)
report_mean_velocity_by_cell(ds, cover)

print("cube,", CUBE)
report_mean_velocity_by_cell_cube(cube_vx, cube_vy, cube_vz)

In [ ]:
plt.hist(cover['velx'].to('km/s').value.flatten(), bins=1000, alpha=0.5, label='plt_cnt_0000')
plt.hist(cube_vx.flatten()/1e5, bins=1000, alpha=0.5, label='mimic 128^3')
plt.ylim(ymin=1e0, ymax=1e4)
plt.yscale('log')
plt.legend()
plt.show()

plt.hist(cover['vely'].to('km/s').value.flatten(), bins=1000, alpha=0.5, label='plt_cnt_0000')
plt.hist(cube_vy.flatten()/1e5, bins=1000, alpha=0.5, label='mimic 128^3')
plt.ylim(ymin=1e0, ymax=1e4)
plt.yscale('log')
plt.legend()
plt.show()

plt.hist(cover['velz'].to('km/s').value.flatten(), bins=1000, alpha=0.5, label='plt_cnt_0000')
plt.hist(cube_vz.flatten()/1e5, bins=1000, alpha=0.5, label='mimic 128^3')
plt.ylim(ymin=1e0, ymax=1e4)
plt.yscale('log')
plt.legend()
plt.show()

# Compute power spectrum of velocity fluctuations

For a domain of length $L$ and sampling $\Delta x = L / N$,
FFT frequency is in units of $L$ (i.e., not angular frequency, no units of 2 pi).

Thus, FFT frequency of 1 corresponds to $k = 2\pi/L$.

In [ ]:
from numpy.fft import fftn, fftfreq, fftshift

In [ ]:
def plot_logslope(x0, y0, slope, x1, *args, **kwargs):
    assert x0 > 0
    xarr = np.logspace(np.log10(x0), np.log10(x1), 100)
    yarr = y0 * (xarr/x0)**slope
    return plt.plot(xarr, yarr, *args, **kwargs)

In [ ]:
def compute_psd(arr, mbinfactor=3):
    """compute one-d PSD in k/(2pi) space"""
    
    assert len(arr.shape) == 3
    assert arr.shape[0] == arr.shape[1] == arr.shape[2]
    
    mx, my, mz = np.meshgrid(
        fftfreq(arr.shape[0], d=1./arr.shape[0]),
        fftfreq(arr.shape[1], d=1./arr.shape[1]),
        fftfreq(arr.shape[2], d=1./arr.shape[2])
    )
    m = (mx**2 + my**2 + mz**2)**0.5
    
    ftarr = np.fft.fftn(arr)
    psd = np.abs(ftarr)**2
    
    mbin_edges = np.linspace(0, np.amax(m)+1, num=int(mbinfactor*np.amax(m)+1), endpoint=True)
    mbin_centers = (mbin_edges[1:] - mbin_edges[:-1])/2 + mbin_edges[:-1]  # linear space...
    mbin_values = np.zeros(mbin_centers.shape)
    
    mbin_idx = np.digitize(m.flatten(), mbin_edges)

    for p, i in zip(psd.flatten(), mbin_idx):
        mbin_values[i] += p
    
    return mbin_centers, mbin_values

In [ ]:
def plot_psd(k, px, py, pz, kmin, kmax):
    """Input: k range, power in vx,vy,vz
    kmin,kmax = range of expected k for plotting
    """
    plt.plot(k, px, '.-', label='vx')
    plt.plot(k, py, '.-', label='vy')
    plt.plot(k, pz, '.-', label='vz')

    plt.axvline(kmin, color='k', linestyle=':')
    plt.axvline(kmax, color='k', linestyle=':')
    
    # figure out where to put slope-y lines
    k_pin = 10
    p_pin = 2 * px[np.searchsorted(k, k_pin)]
    
    plot_logslope(k_pin, p_pin, -5./3, kmin, '-k', linewidth=0.5, label='-5/3')
    plot_logslope(k_pin, p_pin, -5./3, kmax, '-k', linewidth=0.5)
    
    plot_logslope(k_pin, p_pin, -6./3, kmin, '-r', linewidth=0.5, label='-6/3')
    plot_logslope(k_pin, p_pin, -6./3, kmax, '-r', linewidth=0.5)
    
    plt.legend()
    
    plt.xlabel('k (2pi/Lbox)')
    plt.ylabel('power P(k) dk')
    plt.yscale('log')
    plt.xscale('log')

# Does PSD recover {slope, kmin, kmax} in mock data from `turb_sphere.py`?

In [ ]:
def create_spspace_box(kmin, kmax, NCD, Eslp):
    ssbox = np.zeros(NCD, dtype=complex)

    # first, create uniform complex random spectrum in whole array
    # range of coeffs is (-sqrt(2)/2, sqrt(2)/2), it results in
    # range (-1,1) for coeffiecient magnitudes
    ssbox.real = np.sqrt(2.)*(np.random.rand(NCD[0],NCD[1],NCD[2]) - 0.5)
    ssbox.imag = np.sqrt(2.)*(np.random.rand(NCD[0],NCD[1],NCD[2]) - 0.5)

    # create mask that filters selected modes and gives weights according to the
    # given spectrum (Eslp)
    ax = NCD[0]/2-np.abs(np.arange(NCD[0], dtype=np.float64)-NCD[0]/2)
    ay = NCD[1]/2-np.abs(np.arange(NCD[1], dtype=np.float64)-NCD[1]/2)
    az = NCD[2]/2-np.abs(np.arange(NCD[2], dtype=np.float64)-NCD[2]/2)

    (mx, my, mz) = np.meshgrid(ax,ay,az)

    mask = mx*mx + my*my + mz*mz
    mask[np.where(mask < kmin*kmin)] = 0
    mask[np.where(mask > kmax*kmax)] = 0
    # E(k) ~ k^Eslp; number of modes grows with k^2 => Eslp-2
    # v ~ sqrt(E) => 0.5*Eslp-1, k^2 is in mask => 0.25*Eslp-0.5
    mask[np.where(mask>0)] = mask[np.where(mask>0)]**(0.25*Eslp-0.5)

    ssbox *= mask
    return ssbox

def kolmogorov_vel(NCD, kmin, kmax, Eslp):
    # create velocity field in spectral space (sp_vel[xyz])
    # and FFT it to configuration space (vel[xyz])
    sp_velx = create_spspace_box(kmin, kmax, NCD, Eslp)
    sp_vely = create_spspace_box(kmin, kmax, NCD, Eslp)
    sp_velz = create_spspace_box(kmin, kmax, NCD, Eslp)

    velx = np.fft.ifftn(sp_velx).real
    vely = np.fft.ifftn(sp_vely).real
    velz = np.fft.ifftn(sp_velz).real

    return (velx, vely, velz)

In [ ]:
test_slope = -5./3
test_kmax = 32

for test_kmin in [0.1, 1, 3, 5]:
    vxtest, vytest, vztest = kolmogorov_vel((NBLOCK,NBLOCK,NBLOCK), test_kmin, test_kmax, test_slope)

    # apply fake mask to zero out velocities
    ax = np.linspace(-BOXHW, +BOXHW, NBLOCK)
    ay = np.linspace(-BOXHW, +BOXHW, NBLOCK)
    az = np.linspace(-BOXHW, +BOXHW, NBLOCK)
    (xx, yy, zz) = np.meshgrid(ax, ay, az)
    rr = xx**2+yy**2+zz**2

    vxtest[rr>RADIUS**2] = 0
    vytest[rr>RADIUS**2] = 0
    vztest[rr>RADIUS**2] = 0

    mtest, pxtest = compute_psd(vxtest, mbinfactor=2)
    mtest, pytest = compute_psd(vytest, mbinfactor=2)
    mtest, pztest = compute_psd(vztest, mbinfactor=2)
    
    plot_psd(mtest, pxtest, pytest, pztest, test_kmin, test_kmax)
    plt.ylim(1e-5, 1e0)
    plt.show()
    
    del vxtest, vytest, vztest, ax, ay, az, xx, yy, zz, rr, mtest, pxtest, pytest, pztest

# PSD for the 128^3 cube

In [ ]:
m, px = compute_psd(cube_vx, mbinfactor=2)
m, py = compute_psd(cube_vy, mbinfactor=2)
m, pz = compute_psd(cube_vz, mbinfactor=2)

In [ ]:
plot_psd(m, px, py, pz, 1, 32)
plt.ylim(3e17,3e22)
plt.title(CUBE)
plt.show()

# PSD for the plt_cnt_0000 output file

In [ ]:
m_ds, px_ds = compute_psd(cover['velx'], mbinfactor=2)
m_ds, py_ds = compute_psd(cover['vely'], mbinfactor=2)
m_ds, pz_ds = compute_psd(cover['velz'], mbinfactor=2)

In [ ]:
plot_psd(m_ds, px_ds, py_ds, pz_ds, 1, 32)

if "M3V02BWL10_USM" in INITIAL_PLT:
    plt.ylim(1e18, 1e24)
elif "M5V02AW_bad_IGIMF_start" in INITIAL_PLT:
    plt.ylim(1e17, 1e22)
elif "M3V02AW_USM" in INITIAL_PLT:
    plt.ylim(1e18, 1e23)

plt.title(INITIAL_PLT)
plt.show()

# In M3V02BWL10 the slope looks like -2.  Is this an artifact of higher base refinement?

Answer: probably not.

In [ ]:
if "M3V02BWL10_USM" in INITIAL_PLT:
    
    cover_down = ds.covering_grid(level=NREF-2, left_edge=ds.domain_left_edge,
                                  dims=ds.domain_dimensions*2**(NREF-2))
    
    print("NREF-1 shape", cover['velx'].shape)
    print("NREF-2 shape", cover_down['velx'].shape)
    
    m_dsd, px_dsd = compute_psd(cover_down['velx'], mbinfactor=2)
    m_dsd, py_dsd = compute_psd(cover_down['vely'], mbinfactor=2)
    m_dsd, pz_dsd = compute_psd(cover_down['velz'], mbinfactor=2)

In [ ]:
if "M3V02BWL10_USM" in INITIAL_PLT:
    plt.plot(m_dsd, px_dsd, '.-', label='velx')
    plt.plot(m_dsd, py_dsd, '.-', label='vely')
    plt.plot(m_dsd, pz_dsd, '.-', label='velz')

    plt.axvline(1, color='k',alpha=0.5)
    plt.axvline(32, color='k',alpha=0.5)
    
    plot_logslope(1, 1e21, -5./3, 64, '-k', label='-5/3')
    plot_logslope(1, 1e21, -6./3, 64, '-r', label='-6/3')
    plt.ylim(1e17, 1e23)

    plt.title(INITIAL_PLT)
    plt.legend()
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('Frequency (Lbox^-1)')
    plt.ylabel('Power P(k) dk')
    plt.show()